# **GNU Colab**
 
*Here we set out an environment in which you can run persistent desktop systems*.
 
This notebook lets you **run desktop programs** (through many means).
 
At this moment, the notebook is preconfigured to run either **MATE** or **XFCE**.
 
***Expand only if you want to know how this works under the hood.***

In [ ]:
!uptime

## User management

*In here there are functions to manage users.*

Essentially because Colab machines don't have users and we don't want to run everything as root.

***Expand only if you want to know how this works under the hood***.

In [ ]:
#@title New **user** (default: `user`) { vertical-output: true }
run = False #@param {type:"boolean"}
#@markdown *Concise. It also enables `cron` service.*

#@markdown **Do not change with your name, you will have a chance to add a personal user**
#@markdown **from inside the system**.
user = "user" #@param {type:"string"}
password = "testone" #@param {type:"string"}
root = True #@param {type:"boolean"}

from os import environ

def set_hostname():
  hostname = !hostname
  if len(hostname[0]) < 15:
    !hostname gnu-colab-$(hostname)

def create_user(user=user, password=password, root=root):

  from os import listdir as ls
  if user in ls("/home"):
    print("user {} already present".format(user))
    return

  print("Creating user {}".format(user))

  record(user, "user")
  record(password, "password")

  # Add user
  !useradd $user > /dev/null 2>&1
  !usermod -aG sudo user
  !echo -e "$password\n$password" | passwd root > /dev/null 2>&1

  # Directories
  !mkdir /home/$user > /dev/null 2>&1
  !chmod -R 700 /home/$user
  !mkdir -p /home/$user/Projects > /dev/null 2>&1
  !chown -R $user:$user /home/$user
  install("xdg-user-dirs", system='apt')
  !runuser -l $user -c "xdg-user-dirs-update"

  !mkdir -p /tmp/$user > /dev/null 2>&1
  !chown -R $user:$user /tmp/$user 
 
  # Software related
  !touch /var/log/pip.log
  !chown user:user /var/log/pip.log

  # Set root password
  if root:
    !echo -e "$password\n$password" | passwd root

  set_hostname()
  return True


if run:
  create_user()

In [ ]:
#@title Get amenities { vertical-output: true }
run = False #@param {type:"boolean"}
#@markdown - ### Shell
#@markdown I heard you want both `user` and root to use **ohmyzsh**, am I right? 
user = "user" #@param {type:"string"}
root = True #@param {type:"boolean"}
 
#@markdown - ### Window manager
#@markdown I mean `screen`.

#@markdown  - ### Monitoring 
#@markdown I/O, cron, ssh, logging, gpg.

#@markdown - ### Text editor
#@markdown Something *reasonable*
editor = "vim" #@param ["vim", "nano"] {allow-input: true}
 
 #@markdown Install HTTP monitoring?
cockpit = False #@param {type:"boolean"}


from os import environ
def record(variable, env_name): 
  if variable: 
    environ[env_name] = variable
    return True
 
def clone_gnu_colab(user=user):
  !runuser -l $user -c "git clone https://github.com/tallero/gnucolab ~/Projects/gnucolab" > /dev/null 2>&1

def install_oh_my_zsh(user=user, root=root):
  from os.path import exists
  if exists("/home/{}/.oh-my-zsh"):
    print(" oh my zsh already installed")
    return
  oh_my_zsh = "https://raw.githubusercontent.com/ohmyzsh/ohmyzsh/master/tools/install.sh"
  record(oh_my_zsh, "oh_my_zsh")
  !runuser -l $user -c "$(curl -fsSL $oh_my_zsh) --unattended" > /dev/null 2>&1


  if root:
    !sh -c "$(curl -fsSL $oh_my_zsh) --unattended" > /dev/null 2>&1
    !usermod --shell /usr/bin/zsh root > /dev/null 2>&1
 
 
def setup_text_editor(editor=editor):
  install(editor, system='apt')
 
def setup_monitoring(cockpit=cockpit):
 
  # Enable Ubuntu GPG key server

  !timeout 20s apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 51716619E084DAB9 > /dev/null 2>&1
 
  # Enable cron logging
  !sed -i '/cron\./s/^#//g' /etc/rsyslog.d/50-default.conf
 
  if cockpit:
    packages = ("cockpit cockpit-ws cockpit-packagekit "
                "cockpit-docker cockpit-machines cockpit-dashboard "
                "cockpit-system")
    install(packages, system='apt')

def setup_shell(user=user, root=root):
  print("Shell setup")
  record(user, "user")

  !usermod --shell /usr/bin/zsh user > /dev/null 2>&1

  packages = "autossh cron fortune gnupg nyancat nyancat-server openssh-server tor rsyslog screen sysstat zsh"
  install(packages, system='apt')
    
  install_oh_my_zsh()

  clone_gnu_colab()

  setup_text_editor()

  setup_monitoring()

 
  # Restart services
  !service cron restart > /dev/null 2>&1
  !service rsyslog restart > /dev/null 2>&1
  !service ssh restart > /dev/null 2>&1

if run:
  setup_shell()

### Debug
Have fun!

In [ ]:
#@title Start a shell { vertical-output: true }
run = False #@param {type:"boolean"}
#@markdown *Why not?*
user = "root" #@param ["user", "root"] {allow-input: true}
shell = "bash" #@param {type:"string"}


def start_shell_as_user(user=user, shell=shell):
  from os import environ
  environ['user'] = user
  environ['shell'] = shell

  if user == "root":
    !$shell
  if shell == "zsh":
    !su - $user
  
if run:
  start_shell_as_user()

In [ ]:
#@title Show home directory
run = True

def show_home():
  %sx zsh -c "ls /home/user"

if run:
  show_home()

## Persistence

*In here we have functions to ensure persistence.*

We define a `restore` function and a `backup` cron job.

### Routines

In [ ]:
#@title Restore a backup from tar.gz
run = False #@param {type:"boolean"}

user = "user" #@param {type:"string"}
archive = "/home/user/drive/home.tar.gz" #@param {type:"string"}

def restore_from_tar_gz(user=user, archive=archive):
  record(user, "user")
  record(archive, "archive")
  filename = path_leaf(path=archive)
  record(filename, "filename")
  !runuser -l $user -c "cp -r $archive /home/$user/.$filename" 
  !tar -xpzf /home/$user/.$filename -C / --numeric-owner > /dev/null 2>&1
  !rm /home/$user/.$filename

if run:
  restore_from_tar_gz()

In [ ]:
#@title Setup **tunnel device** (user level) { vertical-output: true }
run = False #@param {type:"boolean"}
#@markdown *Create a tunnel device for user `user`*.
user = "user" #@param {type:"string"}
#@markdown (Actually I don't remember what this was for.)

def setup_tunnel_device(user=user):
  record(user, "user")
  !ip tuntap add name tun0 mode tun user $user
  !ip address add 192.0.2.10/24 dev tun0
  !ip link set dev tun0 up

if run:
  setup_tunnel_device()

In [ ]:
#@title Simple Keep Alive (12 hours) { vertical-output: true }
#@markdown *Press this button from the desktop environment to keep this machine alive for 12 hours.*
run = False #@param {type:"boolean"}
user = "user" #@param {type:"string"}
!apt install xdotool xattr > /dev/null 2>&1
from time import sleep

from os import environ
environ['user'] = user

if run:
  while True:
    "Starting Up and down"
    !runuser -l user -c "DISPLAY=:1.0 xdotool key 'Up'"
    sleep(10)
    !runuser -l user -c "DISPLAY=:1.0 xdotool key 'Down'"
    sleep(10)

### Run

In [ ]:
#@title Restore { vertical-output: true }
#@markdown *Restore a gnu colab system.*
run = False #@param {type:"boolean"}

user = "user" #@param {type:"string"}
method = "Google Drive" #@param {type:"string"}

def restore(user=user, method="Google Drive"):
  from os.path import exists

  if method == "Google Drive":
    base_path = "/home/{}/drive/{}.tar.gz"

  else:
    return False

  archives = ["home", "etc", "var.lib.tor"]
  for archive in archives:
    path = base_path.format(user, archive)
    record(path, "path")
    archive_exists = !runuser -l $user -c "test -f $path && echo $path"
    if archive_exists:
      print("Extracting {}".format(archive))
      restore_from_tar_gz(user=user, archive=path)

  !service cron restart > /dev/null 2>&1
  !service rsyslog restart > /dev/null 2>&1
  !service ssh restart > /dev/null 2>&1
  !service tor restart > /dev/null 2>&1

if run:
  restore()

In [ ]:
#@title Backup home, etc to files (`/home/user/drive/`) { vertical-output: true }

#@markdown *A backup cron job for `user`'s home and other funny things is set.*

run  = False #@param {type:"boolean"}

busy  = False


user = "user" #@param {type:"string"}
#@markdown Every:
minutes =  5#@param {type:"integer"}
#@markdown of every
hours = 1 #@param {type: "integer"}

debug  = False #@param {type:"boolean"}


def backup(user=user, hours=hours, minutes=minutes):

  record(user, "user")

  if hours == 0 or minutes == 0:
    print("values have to be positive")
    return
  excluded_paths = ["/home/{}/drive".format(user),
                    "/home.tar.gz",
                    "/etc.tar.gz",
                    "/var.lib.tor.tar.gz",
                    "/home/{}/home.tar.gz".format(user),
                    "/home/{}/etc.tar.gz".format(user),
                    "/home/{}/var.lib.tor.tar.gz".format(user),
                    "/home/{}/.config/Google".format(user)]

  add_exclude = lambda path: "--exclude={}".format(path)

  exclude_command = " ".join([add_exclude(path) for path in excluded_paths])

  chown = lambda user, path: "chown {}:{} {}".format(user, user, path)

  chown_home = chown(user, "/home.tar.gz")
  chown_etc = chown(user, "/etc.tar.gz")
  chown_tor = chown(user, "/var.lib.tor.tar.gz")

  move_home_backup = lambda user: "runuser -l {} -c 'mv /home.tar.gz /home/{}/drive/home.tar.gz'".format(user, user)
  move_home_user = move_home_backup(user)

  move_etc_backup = lambda user: "runuser -l {} -c 'mv /etc.tar.gz /home/{}/drive/etc.tar.gz'".format(user, user)
  move_etc_user = move_etc_backup(user)

  move_tor_backup = lambda user: "runuser -l {} -c 'mv /var.lib.tor.tar.gz /home/{}/drive/var.lib.tor.tar.gz'".format(user, user)
  move_tor_user = move_tor_backup(user)

  backup_home = ("cd / && tar -cpzf home.tar.gz {} --one-file-system /home/{}"
                 " > /dev/null 2>&1 && {} && {}").format(exclude_command, user, chown_home, move_home_user)

  backup_etc = ("cd / && tar -cpzf etc.tar.gz {} --one-file-system /etc"
          " > /dev/null 2>&1 && {} && {}").format(exclude_command, chown_etc, move_etc_user)

  backup_tor = ("cd / && tar -cpzf var.lib.tor.tar.gz {} --one-file-system /var/lib/tor"
          " > /dev/null 2>&1 && {} && {}").format(exclude_command, chown_tor, move_tor_user)

  cron_job = """
PATH=/usr/sbin:/usr/sbin:/usr/bin:/sbin:/bin

# Backup every {} minutes and {}  everyday

# Home
0-59/{} 0-23/{} * * * root {}

# Etc
0-59/{} 0-23/{} * * * root {}

# Var (tor)
0-59/{} 0-23/{} * * * root {}

""".format(minutes, hours, 
           minutes, hours, backup_home, 
           minutes, hours, backup_etc,
           minutes, hours, backup_tor,)

  save_text_to_file(file="/etc/cron.d/backup", content=cron_job)
  !chmod 644 /etc/cron.d/backup

  !service cron restart > /dev/null 2>&1
  !service rsyslog restart > /dev/null 2>&1

  if debug:
    !cat /etc/cron.d/backup
    !ls /
    !runuser -l $user -c "ls /home/user/drive"

def busy_loop(user=user, minutes=minutes, hours=hours):

  from os import environ
  environ['user'] = user
  from time import sleep

  while True:
   # !runuser -l $user -c "tar -cpzf home.tar.gz --exclude=/home.tar.gz --one-file-system /home/user &"i
   !date
   !echo "backup started"
   !cd / && tar -cpzf home.tar.gz --exclude=/home.tar.gz --exclude=/etc.tar.gz --one-file-system /home/user > /dev/null 2>&1
   !cd / && tar -cpzf etc.tar.gz --exclude=/etc.tar.gz --exclude=/home.tar.gz --one-file-system /etc > /dev/null 2>&1
   !chown $user:$user /home.tar.gz /etc.tar.gz

   !runuser -l $user -c "cp /etc.tar.gz /home/$user/drive/etc.tar.gz"
   print("complete, coming back in {} hours and {} minutes.".format(hours, minutes))
   #!runuser -l $user -c "tar -cpzf etc.tar.gz --exclude=/etc.tar.gz --one-file-system /etc"
   sleep(hours + 60*minutes)

if busy and run:
  busy_loop()

if not busy and run:
  backup()

## Connection
*In here we define functions to open the machine of this notebook to remote SSH connections.*

### Routines

In here there are functions needed to set up connection

In [ ]:
#@title Show onion SSH hostname { vertical-output: true }
run = False #@param {type:"boolean"}
log = False #@param {type:"boolean"}

def get_tor_ssh_hostname(log=log):

  if log:
    !service tor status

  with open('/var/lib/tor/ssh/hostname', 'r') as f:
    return f.read()  

if run:
  print(get_tor_ssh_hostname())

In [ ]:
#@title Setup Tor { vertical-output: true }
#@markdown *Install and configure `Tor`*
run = False #@param {type:"boolean"}
debug = False #@param {type:"boolean"}

def setup_tor_ssh_configuration():
  install("tor", system='apt')

  config_file = "/etc/tor/torrc"
  ssh_hidden_service = "HiddenServiceDir /var/lib/tor/ssh/"
  ssh_port_mapping = "HiddenServicePort 22 127.0.0.1:22"

  ssh = line_in_file(path=config_file, line=ssh_hidden_service)
  binding = line_in_file(path=config_file, line=ssh_port_mapping)

  !mkdir -p /var/lib/tor/ssh > /dev/null 2>&1
  !chown -R debian-tor /var/lib/tor
  !chmod -R 700 /var/lib/tor
  !chmod -R 700 /var/lib/tor/ssh

  if not (ssh and binding):
    record(ssh_hidden_service, "ssh_hidden_service")
    record(ssh_port_mapping, "ssh_port_mapping")
    !echo $ssh_hidden_service >> /etc/tor/torrc
    !echo $ssh_port_mapping >> /etc/tor/torrc
    !service tor restart > /dev/null 2>&1
  else:
    if debug:
      print("tor already configured for incoming ssh connections")
      print(get_tor_ssh_hostname())
  
if run:
  setup_tor_ssh_configuration()

In [ ]:
#@title Init SSH directory { vertical-output: true }
run = False #@param {type:"boolean"}
#@markdown *Create SSH directory for user `user` and `root`.*
user = "user" #@param {type:"string"}
root = True #@param {type:"boolean"}

def new_ssh_dirs(user=user, root=root):
  from os import environ
  environ['user'] = user
  environ['ssh_dir'] = "/home/{}/.ssh".format(user)
  !mkdir -p /home/$user/.ssh > /dev/null 2>&1
  !chmod 700 /home/$user/.ssh
  !chown user:user /home/$user/.ssh
  
  config = """# Onion support
Host *.onion
      proxyCommand ncat --proxy 127.0.0.1:9050 --proxy-type socks5 %h %p

Host google_shell
      Hostname localhost
      Port 6666
      User user
"""

  save_text_to_file(file="/home/{}/.ssh/config".format(user), content=config)
  !chown -R user:user $ssh_dir

  if root:
    !mkdir -p /root/.ssh > /dev/null 2>&1
    !chmod 700 /root/.ssh

if run:
  new_ssh_dirs()

In [ ]:
#@title Setup SSH { vertical-output: true }
run = False #@param {type:"boolean"}
#@markdown We setup `openssh-server`, `autossh`, `tor` and `nmap`.
 
def setup_ssh(run=True):
  print("Setup SSH")
  packages = ["openssh-server", "autossh", "nmap"]
  for package in packages:
    install(package, system='apt')

  ssh_config = """
Host *
    ForwardX11 yes
    ForwardX11Trusted yes
    PasswordAuthentication no
    Tunnel yes
    SendEnv LANG LC_*
    HashKnownHosts yes
    GSSAPIAuthentication yes
"""

  save_text_to_file(file="/etc/ssh/ssh_config", content=ssh_config)

  sshd_config = """
ChallengeResponseAuthentication no
UsePAM yes
X11Forwarding yes
PrintMotd no
AcceptEnv LANG LC_*
Subsystem	sftp	/usr/lib/openssh/sftp-server
"""

  save_text_to_file(file="/etc/ssh/sshd_config", content=sshd_config)

  setup_tor_ssh_configuration()

  new_ssh_dirs()

  !service ssh start > /dev/null 2>&1

if run:
  setup_ssh()

In [ ]:
#@title New SSH key pair
#@markdown If it's your first run you could need to generate keys
run = False #@param {type:"boolean"}
user = "user" #@param ["user"]

def new_pair(user=user):
  from os import environ
  environ['user'] = user
  !runuser -l $user -c 'ssh-keygen'

if run:
  new_pair()

#@markdown #### **Show fingerprint**
#@markdown RSA public key of user `user`
run = False #@param {type:"boolean"}
user = "user" #@param {type:"string"}

def show_fingerprint(user=user):
  from os import environ
  environ['user'] = user
  print("{} ssh public key:".format(user))
  !cat /home/$user/.ssh/id_rsa.pub

if run:
  show_fingerprint()

#### Existing configuration
If it's not your first run you want to retrieve your already existing ssh key pairs, to retrieve from a secure location.
We currently have code for:
- Google drive.

You need to have (in the root of your google drive)  
- `/etc/ssh/` properly configured as in a normal ssh server;
- `/etc/tor/` properly containeing a `torrc` with ssh hidden service enabled;
- `/.ssh/` containing a key called `google` and a `config` file contaning an host called `google` to use as a reverse proxy;
- `/var/lib/tor/ssh/`

##### Open from Google **drive**

###### Routines

In [ ]:
#@title Copy from drive directory { vertical-output: true }
run = False #@param {type:"boolean"}
#@markdown *That's because we mount drive as user.*
user = "user" #@param {type:"string"}
source = "/home/user/drive/.ssh" #@param {type:"string"}
destination = "/home/user/" #@param {type:"string"}

from os import environ

def drive_copy(source=source, dest=destination):
  from os.path import join as path_join
  record(source, "source")
  record(dest, "dest")
  record(user, "user")

  temp = path_join("/home/{}".format(user), ".tmp")
  record(temp, "temp")

  print("wut drive_copy")
  !runuser -l $user -c "mkdir $temp"

  !runuser -l $user -c "cp -a $source $temp"
  !cp -rv $temp $dest

  !runuser -l $user -c "rm -rf $temp"

if run:
  drive_copy()
  !ls /home/user/.ssh

In [ ]:
#@title Import
run = False #@param {type:"boolean"}
#@markdown We import:

key = "google" #@param ["id_rsa", "google"] {allow-input: true}
root = True #@param {type:"boolean"}
user = "user" #@param {type:"string"}

def drive_download(key=key, user=user, root=root):
  record(key, "key")
  record(user, "user")

  #@markdown - user configuration;
  !runuser -l $user -c "mkdir -p /home/$user/.tmp"
  
  !runuser -l $user -c "cp -r /home/$user/drive/.ssh /home/$user"

  !runuser -l $user -c "cp -r '/home/$user/drive/etc' '/home/$user/.tmp'"
  !cp -r /home/$user/.tmp/etc /

  !mv /home/$user/.ssh/$key /home/$user/.ssh/id_rsa
  !mv /home/$user/.ssh/$key.pub /home/$user/.ssh/id_rsa.pub
  !cp /home/$user/.ssh/id_rsa.pub /home/$user/.ssh/authorized_keys

  #@markdown - root user configuration;
  !cp /home/$user/.ssh/id_rsa.pub /root/.ssh/authorized_keys

  # Set permissions
  !chmod 700 /home/$user/.ssh
  !chmod 600 /home/$user/.ssh/id_rsa /home/$user/.ssh/id_rsa.pub
  !chmod 755 /home/$user/.ssh/authorized_keys

  !chown -R $user:$user /home/$user

  #@markdown - tor configuration

  !runuser -l $user -c "cp -r '/home/$user/drive/var' '/home/$user/.tmp'"
  !cp -r /home/$user/.tmp/var /
  !chown -R debian-tor:debian-tor /var/lib/tor

  # Restart services
  !service ssh restart > /dev/null 2>&1
  !service tor restart > /dev/null 2>&1

if run:
  drive_download()

In [ ]:
#@title Export
run = False #@param {type:"boolean"}
#@markdown - SSH daemon configuration;
#@markdown - Tor configuration and onion address private key;
#@markdown - user configuration.
root = True #@param {type:"boolean"}
user = "user" #@param {type:"string"}

def drive_upload(user=user, root=root):
  from os import environ
  environ['user'] = user
  # System-wide SSH config files
  !runuser -l $user -c "mkdir -p /home/$user/drive/etc/ssh"
  !runuser -l $user -c "cp /etc/ssh/sshd_config /home/$user/drive/etc/ssh/sshd_config"
  !runuser -l $user -c "cp /etc/ssh/ssh_config /home/$user/drive/etc/ssh/ssh_config"

  # SSH user directory
  !runuser -l $user -c "cp -r /home/$user/.ssh /home/$user/drive"

  # Tor configuration and private keys
  !runuser -l $user -c "mkdir -p /home/$user/drive/var/lib/tor"
  !runuser -l $user -c "mkdir -p /home/$user/drive/etc/tor"
  !cp -r /var/lib/tor/ssh /home/$user/ssh 
  !chown -R $user:$user /home/$user/ssh
  !runuser -l $user -c "cp -r /home/$user/ssh /home/$user/drive/var/lib/tor/ssh"
  !rm -r /home/$user/ssh
  !runuser -l $user -c "cp -r /etc/tor/torrc /home/$user/drive/etc/tor"

if run:
  drive_upload()

##### Run

In [ ]:
#@title ### **Migration tool**
run = False #@param {type:"boolean"}
mountpoint = "/content/drive" #@param {type:"string"}
action = "Import" #@param ["Import", "Export"]

def migration_tool(mountpoint=mountpoint, action=action, debug=run):
  from os.path import join as path_join

  if action == "Import":
    drive_download()
  if action == "Export":
    drive_upload()

  if debug:
    !service tor start
    show_fingerprint()
    print("ssh onion address:")
    !cat /var/lib/tor/ssh/hostname


if run:
  migration_tool()

### Run

*In here the connection takes place (and reverse connection parameters are set).*

In [ ]:
#@title Set reverse proxy { vertical-output: true }
#@markdown *In here we configure a reverse proxy*

run = False #@param {type:"boolean"}

#@markdown #### **Local configuration** (*the machine on this notebook*)
#@markdown The user which will open the connection:
user = "user" #@param {type:"string"}
#@markdown #### **Remote configuration** (*your computer or a proxy*)
remote_user = "google" #@param {type:"string"}
domain = "extranet.arcipelago.ml" #@param {type:"string"}
port =  58372#@param {type:"integer"}
fingerprint = "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAAD4QDjYrny2zge7VStf11KpzHPioepFZhZHBUDGXdnQuGxB+jrFDji3mTajd94Ktom8zW5RQJkyCTHj/6LnIvG3bN9I3nHVne6bib38EaOIiTV1Dz9Vt0SkJbpgCiYRw/MFmA3w/pQ90c4K7V2wnFyuea6IWbqPdhMhiDVXXuaEcSxx91U2fYLAPweBLY5Y6HODIGCJwxrvY8pzLYMMOWYU4dniwyzWT+QRUQNNq7sP403XVPJ40Qg5c9YUlu7u1/YeO8aiYxg0pZ5yKbAO2dvPMrHd1wUXyNL80YfobgvawZCO09GE0iuC1YFgC3vo/b1eacEctMGryUQuQqTRBgbPq4LviPwyHcQMXerIThVLCU7FPiuwt1ylxxkjOEXyoZyAs30/P84R3M0vhxY4YF+h5NloIpwTWdYQhNvk5zQLthqymeHlWDl82glStue9P22pmm2axjqQ6ezOZ+frkqxlhPzN1kCxlaWCv1rpTCmbDOWL0GlPD6UZVWfBS9QcRYpLtqAnuQfxRK8U0/XAo5nJPlHIfjWRdh/cXnJlqLQJ13G78urmmS5G9RmyNRdOEi+jIF8sGjSkuppKhh1yQGbYJ8iVm1XTgZph4+lqTwbcWiOHSMQm5Ur+DvM6ElpQICTNnlEBGp9cdAY2fInVQcMWyYzrx7W2L7/sPBB+4hjvjudoPyh5J9UOkeF61yPN8gODM7wBow91VDmLbferbRTYIlRBmHYoVngSx4md/D+3bb5AKMOAksK42ruGgGS9cN6XAm3D2wKBxC0IaqfRB2L1IPuPPMjAAaljciHY0wD2o0gnmYusyS+TO1fdpsMV99qxbAZUNQ1Y4sN5NI7MYNN/AcbgssYTVuLrWLhdHmfTlrMcXlNDgboRKpMUsbEEyHdwcSDs5SmW61/ObGsXUi9/ZYYGXr0F76Mkbd3X4PXy1Ze65Eevc5eV699Bn0tCHrrILCrUt6n29Vc4Q4ysVs45S1vB524w4wWRfBPf2+Kk+YcSbN4Dt1Y++3onlLRVxVJaYfSRwyiALqSiYj16h7Aoc4bvF5icmoaiLVBJ7fvMTDN7Ach3Tve5DMigca2D2mlb4rOG1D4YiwBj/chRhOFe1vK5cfiy8FWIXPcYnWPluPyWGGgiGqqkdSuEDb4c/ZX2DCkUWMzGh1ZSa+EML+6OvXFDK/cz41Les+Ot5D2sbWu/fiTKeESVEKcwpXDjRHcBYoY/nbXZeQ6LKaU8AHqn1F15ci9M0wQLSscn86SNf3fuskebsYRPMLd5agJh6O/SGa7IKBSS9k1xzcKxYCZ2jITyF9oItlP9A+gNTeny6ju2Q== colab@gnu" #@param {type:"string"}

debug = True #@param {type:"boolean"}

def create_ssh_config(user=user, remote_user=remote_user, 
                      domain=domain, port=port, fingerprint=fingerprint,
                      interactive=False, debug=debug):

  from os.path import join as path_join
  record(user, "user")
  ssh_config_path = path_join("/home", user, ".ssh/config")
  record(ssh_config_path, "ssh_config_path")
  authorized_keys = path_join("/home", user, ".ssh/authorized_keys")
  record(authorized_keys, "authorized_keys")
  record(fingerprint, "fingerprint")

  while ssh_config_path:
    ssh_config_exists = !runuser -l $user -c "test -f $ssh_config_path && echo True"
    if ssh_config_exists:

      message = "configuration already existing, do you want to overwrite it? [y/N] "
      answer = input(message)
      if answer.lower() == "y":
        !cat /home/$user/.ssh/config
        pass
      else:
        break

      if debug:
        print("ssh config:")
        !cat /home/$user/.ssh/config

    if interactive:

      message = "Please insert the domain of the machine to connect to:\n"
      domain = input(message)
      if not domain:
        message = ("Please insert an address if you select this option. "
                                "Connection and graphical environment won't work atm.")
        print(message)
        raise Exception
  
      message = "Please insert the ssh port: (default: 22)\n"
      port = (input(message) or 22)

      message = "Please insert the name of the user you will connect to: (default: google)\n"
      remote_user = (input(message) or "google")

    config = """# Onion support
Host *.onion
      proxyCommand ncat --proxy 127.0.0.1:9050 --proxy-type socks5 %h %p

Host google
      HostName {}
      Port {}
      User {}

Host google_shell
      Hostname localhost
      Port 6666
      User user
""".format(domain, port, remote_user)

    save_text_to_file(file=ssh_config_path, content=config)
    ssh_config_path = None

  while authorized_keys:

    def prompt():
      message = "Please insert the fingerprint of the SSH key you will use to connect to this machine\n"
      fingerprint = input(message)
      if not fingerprint:
        print("You need a fingerprint, otherwise you won't be able to connect to this machine through SSH!")
        raise Exception
      return fingerprint

    authorized_keys_exists = !runuser -l $user -c "test -f $authorized_keys && echo True"

    if authorized_keys_exists:
      print("authorized keys:")
      !cat /home/user/.ssh/authorized_keys

      if interactive:
        overwrite = input("Add new one? [y/N]")
        if overwrite.lower() == "y":
          fingerprint = prompt()
      
      is_fingerprint_in = line_in_file(path=authorized_keys, line=fingerprint)
      if not is_fingerprint_in:
        !runuser -l $user -c "echo $fingerprint >> $authorized_keys"

    else:
      if interactive: 
        fingerprint = prompt()
      !runuser -l $user -c "echo $fingerprint > $authorized_keys"

    if debug:
      print("authorized_keys:")
      !cat /home/$user/.ssh/authorized_keys

    break

if run:
  create_ssh_config()

In [ ]:
#@title Connect { vertical-output: true }
run = False #@param {type:"boolean"}
 
#@markdown Select this option if you want to enable *root* access.
root = False #@param {type:"boolean"}
#@markdown Specify the user do you want to access to.
user = "user" #@param {type:"string"}
#@markdown Enable if you wish to assign an onion address to this machine.
tor = True #@param {type:"boolean"}
#@markdown Enable if you wish to connect to the machine through a reverse proxy (see above).
reverse = True #@param {type:"boolean"}
local = 22 #@param {type:"integer"}
remote = 6666 #@param {type:"integer"}
debug = True #@param {type:"boolean"}

def connect(root=root, user=user, tor=tor, 
            reverse=reverse, local=local, remote=remote,
            fingerprint=fingerprint, debug=debug):
  record(user, "user")
  record(str(local), "local_port")
  record(str(remote), "remote_port")
  config_path =   "/home/{}/.ssh/config".format(user)
  key_path = "/home/{}/.ssh/id_rsa".format(user)
  record(key_path, "key_path")
  record(config_path, "config_path")
  
  are_there_keys = !runuser -l $user -c "test -f $key_path && echo True"
  are_there_root_keys = !test -f /root/.ssh/id_rsa.pub && echo True
 
  if not are_there_keys:
    new_pair(user)
  if root:
    if not are_there_root_keys:
      new_pair("root")
  if not are_there_keys or not are_there_root_keys:
    migration_tool(mountpoint=mountpoint, action="Export")
 
  print("""From any computer: you can connect to
 {}@{}""".format(user, get_tor_ssh_hostname()))
 
  if reverse:
    message = "Show the key you have to authorize on your proxy computer? [Y/n] \n"
    if "y" == input(message).lower():
      !cat $key_path
      !echo $key_path
    
    #!ssh -tt -o StrictHostKeyChecking=no -i $key_path -F $config_path -R $remote_port:localhost:$local_port google ssh google_shell

    record("/home/$user/autossh.log", "AUTOSSH_LOGFILE")
    #!autossh -N -M 10984 -f -o 'PubkeyAuthentication=yes' -o 'PasswordAuthentication=no' -o StrictHostKeyChecking=no -F $config_path -i $key_path -R $remote_port:localhost:$local_port google
    !runuser -l $user -c "autossh -N -M 10984 -f -o 'PubkeyAuthentication=yes StrictHostKeyChecking=no' -F $config_path -i $key_path -R $remote_port:localhost:$local_port google"
    
    if debug:
      !cat /home/$user/autossh.log
    print("""From the reverse proxy: you can connect to
     {}@localhost, port {}""".format(user, remote))
    
if run:
  connect()

### Debug

*In here there are functions to discover what's going on while trying to setup availability for remote connections.*

In [ ]:
#@title Test SSH connection
#@markdown It connects to hostname `google`. 
start = False #@param {type:"boolean"}

if start:
  !ssh -o StrictHostKeyChecking=no -F /home/user/.ssh/config -i /home/user/.ssh/id_rsa google
  # Verboso
  # !ssh -i /home/user/.ssh/id_rsa -o StrictHostKeyChecking=no -p 58372 google@extranet.arcipelago.ml

In [ ]:
#@title Test SSH reverse tunnel
#@markdown it opens a reverse tunnel to hostname `google`.
start = False #@param {type:"boolean"}


if start:
  !ssh -o StrictHostKeyChecking=no -F /home/user/.ssh/config -i /home/user/.ssh/id_rsa -R 6666:localhost:22 google

In [ ]:
#@title Show ssh config
#@markdown it shows the ssh configuration file for an user
user = "user" #@param {type:"string"}
#@markdown (activate the switch before running it)
start = 1 #@param {type:"slider", min:0, max:1, step:1}

from os import environ

environ['user'] = user

if start:
  if user != "root":
    !cat /home/$user/.ssh/config
  if user == "root":
    !cat /root/.ssh/config

In [ ]:
#@title Show SSH directory
#@markdown show the content of the `.ssh` directory of an user.

user = "user" #@param {type:"string"}
print_tty = False #@param {type:"boolean"}
start = False #@param {type:"boolean"}
variable_name = False
variable_name = ""

if start:
  if user != "root":
    !ls -lsh /home/user/.ssh
    !du /home/user/.ssh
    !stat /home/user/.ssh
  if user == "root":
    !ls -lsh /root/.ssh
    !du /root/.ssh
    !stat /root/.ssh

In [ ]:
#@title Show `user` known_hosts
start = False #@param {type:"boolean"}
if start:
  !cat /home/user/.ssh/known_hosts

In [ ]:
#@title Copy `user` SSH keys on proxy
start = False #@param {type:"boolean"}

if start:
  !scp -i /home/user/.ssh/id_rsa -o StrictHostKeyChecking=no -F /home/user/.ssh/config /home/user/.ssh/id_rsa* google:~/.ssh/

In [ ]:
#@title Check if /dev/tty exists
run = False #@param {type:"boolean"}

if run:
  !ls -la /dev/tty

In [ ]:
#@title Show private key of user `user`
run = False #@param {type:"boolean"}

if run:
  !cat /home/user/.ssh/id_rsa

## Web Services

*In here there are functions for interacting with various web services.*

#### Google Drive Filesystem

*In here we mount your drive in `user` home.*

In [ ]:
#@title As user { vertical-output: true }
#@markdown *Check sign-in this cell to sign-in as `user`.*
run = False #@param {type:"boolean"}
#@markdown In here a shortcut to your google drive is inserted inside
#@markdown `user` home. It is read-only for `user`.
user = "user" #@param {type:"string"}
debug = False #@param {type:"boolean"}
mountpoint = "/content/drive"

from os.path import join as path_join
from os import listdir as ls

def drive_mount(user=user):
  user_home = "/home/{}".format(user)
  drive_mount_bin = path_join(user_home, "drive_mount.sh")

  record(user, "user")
  record(user_home, "user_home")
  record(drive_mount_bin, "drive_mount_bin")

  mounts = %sx mount
  mounted = "drive on /content/drive type fuse.drive"
  if any(mount.startswith(mounted) for mount in mounts):
    print("Drive already mounted.")
    return
    
  install("google-colab", system='pip', user=user)

  def setup_user_mount(user=user, debug=debug):
    !cp -r /root/.config/Google /home/$user/.config/Google > /dev/null 2>&1
    !mkdir -p /home/$user/.config/Google/DriveFS/Logs
    !chown -R $user:$user /home/user/.config/Google
    !mkdir -p /content /drive > /dev/null 2>&1
    !chown -R $user:$user /content/.config
    !chown -R $user:$user /content > /dev/null 2>&1

  setup_user_mount()

  mount = """
#!/bin/sh

from os import environ as env
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'

def drive_mount(mountpoint="/content/drive"):
  try:
    drive.mount(mountpoint)
  except Exception as e:
    raise e

if __name__ == "__main__":
  drive_mount()
"""
  
  save_text_to_file(file=drive_mount_bin, content=mount)

  if debug:
    print("hello")
  
  !chown -R  $user:$user $user_home
  !chmod u+x $drive_mount_bin

  !runuser -l $user -c "python3 /home/$user/drive_mount.sh"

  !ln -s /content/drive/My\ Drive $user_home/drive > /dev/null 2>&1
  !chown $user:$user $user_home/drive > /dev/null 2>&1

if run:
  drive_mount()

##### Debug
You won't have fun in here.

In [ ]:
#@title As administrator
#@markdown *Check sign-in this cell to sign-in as `root`.*
run = False #@param {type:"boolean"}
mountpoint = "/content/drive" #@param {type:"string"}


def drive_mount_root(mountpoint='/content/drive'):
  from os.path import join
  from google.colab import drive
  drive_root_directory = join(mountpoint, "My Drive")
  try:
    drive.mount(mountpoint)
  except Exception as e:
    raise e
  
if run:
  drive_mount_root()

#### Amazon Web Services

In [ ]:
#@title Amazon integration
run = False #@param {type:"boolean"}

def setup_amazon():
  !apt install aws-shell > /dev/null 2>&1
  !pip3 install boto3 > /dev/null 2>&1

if run:
  setup_amazon()

### Ngrok
[TODO](https://github.com/tallero/GNUColab/issues/3)

## Boot

Actual boot function.

### Routines

*Expand only if you want to know how this works under the hood*.

#### Notebook loader

In [ ]:
#@title Imports
import io, os, sys, types
 
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [ ]:
#@title `find_notebook(fullname, path=None)`
#@markdown *find a notebook, given its fully qualified name and an optional path*

#@markdown This turns `"foo.bar"` into `"foo/bar.ipynb"`
#@markdown and tries turning `"Foo_Bar"` into `"Foo Bar"` if `Foo_Bar`
#@markdown does not exist.

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [ ]:
#@title `NotebookLoader(object)`
#@markdown Module Loader for Jupyter Notebooks

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [ ]:
#@title `NotebookFinder(object)`
#@markdown Module finder that locates Jupyter Notebooks

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [ ]:
#@title `sys.meta_path.append(NotebookFinder())`

sys.meta_path.append(NotebookFinder())

#### Utils

In [ ]:
#@title Make so that if { vertical-output: true }

run = True #@param{type:"boolean"}

from os import listdir as ls
#@markdown If the boolean `bad_condition` happens, `a_function(error, *args)` gets executed.

#@markdown i.e:
#@markdown ```python3
#@markdown empty_root = len(ls("/")) == 0
#@markdown
#@markdown def scream(error=None):
#@markdown   print("ARGH!")
#@markdown ```
empty_root =  len(ls("/")) == 0

def scream(error=None):
  print("ARGH!")

bad_condition =  empty_root #@param {type:"raw"}
a_function = scream #@param {type:"raw"}

def make_so_that_if(bad_condition=bad_condition, 
                    a_function=a_function, 
                    *a_function_arguments):
  try:
    assert not bad_condition
    return True
  except AssertionError as e:
    a_function(e, *a_function_arguments)

if run:
  make_so_that_if()

In [ ]:
#@title Record { vertical-output: true }
#@markdown *Put a python string inside a bash environment variable* (`os.environ` wrap)

#@markdown  ```python3
#@markdown record(variable, "a_bash_variable_name")
#@markdown ```
run = True #@param{type:"boolean"}
variable = "a python string" #@param {type:"raw"}
env_name = "a_bash_variable_name" #@param {type:"string"}

from os import environ
def record(variable=variable, env_name=env_name): 
  if variable: 
    environ[env_name] = variable
    return True

if run:
  record()
  print("${} = {}".format(env_name, environ[env_name]))

In [ ]:
#@title Save text to text file
run = True #@param {type:"boolean"}
#@markdown *Because.*
file = "/examplefile" #@param {type:"string"}
content = "content of the file" #@param {type: "string"}


def save_text_to_file(file=file, content=content):
  with open(file, "w") as f:
    f.write(content)

if run:
  save_text_to_file()
  
  record(file, "file")
  print(file)
  print("----------------")
  !cat $file

In [ ]:
#@title Search text in a file { vertical-output: true }

run = True #@param{type:"boolean"}

#@markdown *Same reason.*
path = "/examplefile" #@param{type:"string"}
pattern = "the thing to search for" #@param {type:"string"}

def search_in_file(path=path, pattern=pattern):
  with open(path, "r") as f:
    content = f.read()
    
    if pattern in content:
      return True

if run:
  print(search_in_file())

In [ ]:
#@title Replace text in a file { vertical-output: true }

run = False #@param{type:"boolean"}

#@markdown *Not having had these kinds of graphical macros before makes me wonder.*

path = "/examplefile" #@param{type:"string"}
pattern = "the thing to replace" #@param {type:"string"}
replacement = "the replacement" #@param {type:"string"}

def replace_in_file(path=path, pattern=pattern, replacement=replacement):
  record(path, "file_path")
  
  with open(path, "r") as f:
    content = f.read()
    
  content = content.replace(pattern, replacement)
 
  with open(path, "w") as f:
    f.write(content)

if run:
  replace_in_file()
  !cat $file_path

In [ ]:
#@title  Check line { vertical-output: true }
run = True #@param{type:"boolean"}
#@markdown *Check if there is a line in a file.*
path = "/examplefile" #@param{type:"string"}
line = "line to check" #@param {type:"string"}

def line_in_file(path=path, line=line):
  with open(path, "r") as f:
    content = f.read()
    
  content = content.split("\n")
  if line in content:
    return True

if run:
  event = ""
  if not line_in_file() :
    event = "not"
  print("{} is {} in {}".format(line, event, path))

In [ ]:
#@title Append text to a file

run = True #@param{type:"boolean"}

#@markdown *Let's append text to files.*

file_path = "/examplefile" #@param{type:"string"}
text = "the text to append" #@param {type:"string"}

def append_to(error=None, file_path=file_path, text=text):
  record(user, "user")
  record(text, "text")
  record(file_path, "file_path")
  !runuser -l $user -c "echo $text >> $file_path"

if run:
  append_to()
  !cat $file_path

In [ ]:
#@title Path leaf
run = True #@param {type:"boolean"}

#@markdown *Extract filename from path.*

path = "/home/user/.ssh/id_rsa.pub" #@param {type:"string"}

from ntpath import basename
from ntpath import split as nt_split

def path_leaf(path=path):
  head, tail = nt_split(path)
  return tail or basename(head)

if run:
  path_leaf()

In [ ]:
#@title URL Path leaf { vertical-output: true }
run = True #@param {type:"boolean"}

#@markdown *Extract filename from url.*

url = "http://colab.gnu/resource" #@param {type:"string"}

def url_path_leaf(url=url):
  from os.path import basename
  from urllib.parse import urlparse

  a = urlparse(url)
  return basename(a.path)

if run:
  print(url_path_leaf())

In [ ]:
#@title Enable lots of Ubuntu repositories
run = False #@param{type:"boolean"}

#@markdown *Potentially very dangerous.*

def enable_many_ubuntu_repos():
  print("Enabling universe, multiverse, restricted repositories")
  
  # Enable them
  !add-apt-repository universe > /dev/null 2>&1
  !add-apt-repository multiverse > /dev/null 2>&1
  !add-apt-repository restricted > /dev/null 2>&1

  # Enable i386 arch
  !dpkg --add-architecture i386 > /dev/null 2>&1
  !apt update > /dev/null 2>&1

if run:
  enable_many_ubuntu_repos()

In [ ]:
#@title Package management { vertical-output: true }
run = False #@param {type:"boolean"}
#@markdown *Package management actions (gasp)*
action = "Install" #@param ["Install", "Show installed files", "Remove"] {allow-input: true}

package = "pgpgram" #@param {type:"string"}
system = "pip" #@param ["apt", "pip", ""]
user = "user" #@param {type:"string"}
debug = False #@param {type:"boolean"}

def install(package=package, system=system, action=action, user=user, debug=debug):
  if debug:
    print("installing {}".format(package))
  record(package, "package")
  record(user, "user")
  action = action.lower()
  record(action, "action")

  if system == "apt":
    !apt --fix-broken install > /dev/null 2>&1
    !killall apt > /dev/null 2>&1
    !rm /var/lib/dpkg/lock-frontend
    !dpkg --configure -a > /dev/null 2>&1

    !apt-get  install -o Dpkg::Options::="--force-confold" --no-install-recommends --allow -y $package >/dev/null 2>&1
    
    !dpkg --configure -a > /dev/null 2>&1 
    !apt  update > /dev/null 2>&1

    !apt install $package > /dev/null 2>&1

  if system == "pip":
    if action == "remove":
      record("uninstall", "action")
    !runuser -l $user -c "python3 -m pip $action --user $package" > /dev/null 2>&1

  if system == "npm":
    install("nodejs", system='apt')
    install("npm", system='apt')
    install("node-gyp", system='apt')
    install("nodejs-dev", system='apt')
    install("libssl1.0-dev", system='apt')

    package
    record("/home/{}/Projects/destreamer".format(user), "project_path")

    !runuser -l $user -c "mkdir -p $project_path"
    !runuser -l $user -c "git clone https://github.com/snobu/destreamer $project_path" > /dev/null 2>&1

    !runuser -l $user -c "cd $project_path && npm install" > /dev/null 2>&1

    !runuser -l $user -c "cd $project_path && npm run build" > /dev/null 2>&1

def get_package_files(package=package):
  from os import environ
  environ['package'] = package
  files = !dpkg-query -L $package
  return files

def remove(package=package, system='pip'):
    if system == "pip":
      record("uninstall", "action")
    !runuser -l $user -c "python3 -m pip $action --user $package"

if run:
  if action == "install":
    install()
  if action == "Show installed files":
    from pprint import pprint
    pprint(get_package_files())
  if action == "Remove":
    remove()

### Debug

#### System information

In [ ]:
#@title Are we in docker? { vertical-output: true }
run = True #@param {type:"boolean"}

def in_docker():
  """ Returns: True if running in a Docker container, else False """
  with open('/proc/1/cgroup', 'rt') as ifh:
      return 'docker' in ifh.read()

if run:
  if in_docker():
    print("We are in a docker container.")

In [ ]:
#@title Are we in a privileged docker? { vertical-output: true }
run = True #@param {type:"boolean"}

def can_load_modules():
  modules = ["br_netfilter", 
             "veth",
             "virtio_balloon",
             "aesni_intel"]
  for module in modules:
    record(module, "module")
    output = !rmmod $module
    for o in output:
      if "Operation not permitted" in o:
        print(output)
        return False
  return True

if run:
  if not can_load_modules():
    print("No, since we can't unload kernel modules")

In [ ]:
#@title Get *running* kernel version { vertical-output: true }
run = True #@param {type:"boolean"}

def get_running_kernel_version():
  from os import uname
  return uname().release

if run:
  print(get_running_kernel_version())

In [ ]:
#@title Get *installed* ubuntu kernel modules version { vertical-output: true }
run = True #@param {type:"boolean"}

def get_installed_kernel_modules_version():
  from os import listdir as ls
  modules = ls("/lib/modules")
  if len(modules):
    if len(modules) > 1:
      print("installed modules versions: {}".format(modules))
    return modules[0]

if run:
  print(get_installed_kernel_modules_version())

In [ ]:
#@title Get kernel name in Ubuntu modules' declaration { vertical-output: true }
run = True #@param {type:"boolean"}

def get_running_modules_kernel_version():
  return get_running_kernel_version()[:-1] + "-generic"

if run:
  print(get_running_modules_kernel_version())

In [ ]:
#@title List available kernel modules (`ko` files) { vertical-output: true }

run = True #@param {type:"boolean"}

def list_ko_files():
  !find /lib/modules/ | grep .ko

if run:
  list_ko_files()

## Graphical environment

*In here we install an user-friendly graphical environment to easy advanced tasks and setup a screen sharing program.*

**Manual intervention required**: at first boot, to set VNC password at runtime.

If you enabled Google Drive integration you will find a link to your drive in your home directory.

**XFCE (default)**:
- **Best**: 9 mins
- **Worst**: 12 mins

**MATE**:
- **Best**:
- **Worst**:  4 mins

### Routines

*In here we provide functions to install and configure TigerVNC and additional tools useful in a desktop.*

In [ ]:
#@title Setup TigerVNC { vertical-output: true }
run = False #@param {type:"boolean"}

#@markdown **Set credentials:**
user = "user" #@param {type:"string"}
password = "testone" #@param {type:"string"}


def setup_vnc(user=user, password=password):

  record(user, "user")

  packages = ("dbus-x11 tigervnc-standalone-server tigervnc-xorg-extension xinit "
                           "xserver-xorg-video-dummy x11-xserver-utils xauth")
  install(packages, system='apt')

  !wget http://xpra.org/xorg.conf > /dev/null 2>&1
  !mv xorg.conf /etc/X11

  !runuser -l $user -c 'touch /home/$user/.Xauthority'

  !runuser -l $user -c "touch /home/$user/.Xresources"

  xwrapper_path = "/etc/X11/Xwrapper.config"
  xwrapper = "allowed_users=anybody"
  save_text_to_file(file=xwrapper_path, content=xwrapper)

  !runuser -l $user -c "mkdir -p /home/$user/.vnc"
  !runuser -l $user -c "mkdir -p /home/$user/drive/.vnc"

  vnc_password_path = "/home/{}/.vnc/passwd".format(user)
  record(password, "vnc_password")
  record(vnc_password_path, "vnc_password_path")

  !runuser -l $user -c "echo $vnc_password | vncpasswd -f > $vnc_password_path"

  vnc_config_path = "/home/{}/.vnc/config".format(user)
  record(vnc_config_path, 'vnc_config_path')

  config = """session=xfce
geometry=1920x1080
localhost
alwaysshared"""

  save_text_to_file(vnc_config_path, content=config)

  # Permission check
  !chown -R $user:$user /home/$user

    #!runuser -l $user -c "echo $vnc_config > $vnc_config_path" 
    #!vim $vnc_password_path

  set_vnc_startup_file()

if run:
  setup_vnc()

In [ ]:
#@title Set VNC Startup file
#@markdown *Here we set the VNC startup file*
run = False #@param {type:"boolean"}

#@markdown What session do you want?
session = "mate-session" #@param {type:"string"}

def set_vnc_startup_file(user=user, session=session):
  from os import environ
  environ['user'] = user
 
  # Set VNC startup file
  !mkdir -p /home/$user/.vnc
  !chown $user:$user /home/$user/.vnc

  xstartup = """#!/bin/sh
unset SESSION_MANAGER
unset DBUS_SESSION_BUS_ADDRESS
[ -x /etc/vnc/xstartup ] && exec /etc/vnc/xstartup
[ -r $HOME/.Xresources ] && xrdb $HOME/.Xresources
vncconfig -iconic &
/usr/bin/{}
""".format(session)

  save_text_to_file(file="/home/{}/.vnc/xstartup.new".format(user), content=xstartup)

  !chown $user:$user /home/$user/.vnc/xstartup.new
  !chmod 755 /home/$user/.vnc/xstartup.new
  !runuser -l $user -c "mv /home/$user/.vnc/xstartup.new /home/$user/.vnc/xstartup"
  !chmod 644 /home/$user/.vnc/xstartup


if run:
  set_vnc_startup_file()

In [ ]:
#@title Install Desktop Environment { vertical-output: true }
#@markdown *This lets you install a DE*

run = False #@param {type:"boolean"}

#@markdown What you can choose:
DE = "MATE" #@param ["MATE", "XFCE"]
install_type = "full" #@param ["minimal",  "full"]

def record(variable, env_name): 
  if variable: 
    environ[env_name] = variable
    return True

def de_install(DE=DE, install_type=install_type):
  if DE == "MATE":
    if install_type == "minimal":
      packages = ("mate-panel marco mate-session-manager"
      "mate-control-center" "mate-applets" "fonts-dejavu")
      install(packages, system='apt')
    if install_type == "full":
      install("mate", system='apt')
    session = "mate-session"

  if DE == "XFCE":
    install("xubuntu-desktop", system='apt')
    session = "xfce4-session"

  # Default applications
  setup_applications()

  return session

if run:
  session = de_install()

In [ ]:
#@title Application defaults { vertical-output: true }
run = False #@param {type:"boolean"}

#@markdown **Runtime:** 8 minutes

#@markdown **Re-run:** 1 minute

#@markdown What's a VNC server without a VNC client?
vnc_viewer = "remmina" #@param ["remmina", ""] {allow-input: true}

#@markdown Your web browser of choice (chromium is pre-installed).
web_browser = "qutebrowser" #@param ["firefox", "epiphany-browser", "qutebrowser", "qutebrowser_pip", "epiphany-browser", "chromium-browser"] {allow-input: true}

#@markdown Install an email client
email = "evolution" #@param ["evolution", "thunderbird", ""]

#@markdown Do you want to chat?
chat = True #@param {type:"boolean"}

#@markdown Do you want to play games?
games = "steam" #@param ["steam", "gnome-games", "epsxe", ""]

#@markdown Your favorite media player.
media_player = "mpv" #@param ["mpv", "vlc", "totem"] {allow-input: true}

#@markdown A WSIWYG text editor
text_editor = "gedit" #@param ["thunar", "caja", ""] {allow-input: true}

#@markdown Do you want to edit images?
image_editing = True #@param {type:"boolean"}

#@markdown A file manager
file_manager = "nautilus" #@param ["gnome-builder", "kate", ""] {allow-input: true}

#@markdown Sticky notes
sticky_notes = "tomboy" #@param ["tilix", ""] {allow-input: true}

#@markdown System monitoring (*and Usage and Baobab*)
monitoring = "gnome-system-monitor" #@param [""] {allow-input: true}

#@markdown A terminal emulator
terminal = "tilix" #@param ["tilix", ""] {allow-input: true}

#@markdown A visual package manager
package_manager = "synaptic" #@param ["synaptic", "gnome-packagekit", "gnome-software"] {allow-input: true}

#@markdown A users utility
users = "gnome-system-tools" #@param ["gnome-system-tools", ""]

#@markdown Visual style
style = "gnome" #@param ["gnome", ""] {allow-input: true}

#@markdown Install tweaks
tweaks = True #@param {type:"boolean"}

#@markdown Install dock
dock = False #@param {type: "boolean"}


def install_steam():
  # Enable multiverse
  !add-apt-repository multiverse > /dev/null 2>&1
  # Enable i386 arch
  !dpkg --add-architecture i386 > /dev/null 2>&1
  !apt update > /dev/null 2>&1
  !apt --fix-broken install > /dev/null 2>&1 #install(package=package, system=system):

  # Install zenity
  install("zenity", system="apt")
  # LibGL
  install("libgl1:i386", system="apt")
  # Nvidia thing
  install("libgl1-nvidia-glvnd-glx:i386", system="apt")

 # Get and install steam from pkg
  !cd /tmp && wget https://cdn.cloudflare.steamstatic.com/client/installer/steam.deb > /dev/null 2>&1
  !dpkg -i /tmp/steam.deb > /dev/null 2>&1

!dpkg --configure -a > /dev/null 2>&1

def setup_applications(vnc_viewer=vnc_viewer,
                                              web_browser=web_browser,
                                              email=email,
                                              chat=chat,
                                              games=games,
                                              media_player=media_player,
                                              text_editor=text_editor,
                                              image_editing=image_editing,
                                              file_manager=file_manager,
                                              sticky_notes=sticky_notes,
                                              terminal=terminal,
                                              monitoring=monitoring,
                                              package_manager=package_manager,
                                              users=users,
                                              style=style,
                                              tweaks=tweaks,
                                              dock=dock):
  
  print("Installing desktop applications")

  packages = ""

  if vnc_viewer: packages = packages + " " + vnc_viewer

  if web_browser:
    if web_browser == "qutebrowser_pip":
      packages = packages + " " + "qutebrowser"
    else:
      packages = packages + " " + web_browser

    packages = packages + " " + "chromium-browser"

  if email:
    packages = packages + " " + email

  if chat:
    packages = packages + " " + "gajim telegram-desktop xchat"
    # TODO: Add upstream and fractal

  if record(games, "games"):
    if games == "steam":
      install_steam()
    else:
      install(games, system='apt')

  if record(media_player, "media_player"):
    packages = packages + " " + "pavucontrol"

  if record(file_manager, "file_manager"):
    packages = packages + " " + file_manager

  if text_editor:
    packages = packages + " " + text_editor

  if sticky_notes:
     packages = packages + " " + sticky_notes

  if monitoring:
    packages = packages + " " + monitoring + " gnome-usage baobab file-roller"

  if terminal:
    packages = packages + " " + terminal

  if package_manager:
    packages = packages + " " + package_manager + " gdebi"

  if record(users, "users"):
    packages = packages + " " + users + " intltool"

  if record(style, "style"):
    if style == "gnome":
      style = ("gnome-icon-theme adwaita-icon-theme "
      "adwaita-icon-theme-full "
      "gnome-themes-standard fonts-cantarell "
      "ttf-dejavu-core ttf-dejavu-extra")
  
    packages = packages + " " + style

  if dock:
    install_dock()

  if tweaks:
    tweak_packages = ("gnome-tweak-tool mate-tweak " 
      "mate-dock-applet libnotify-bin dconf-editor "
      "xdg-user-dirs-gtk")
    packages = packages + " " + tweak_packages

  install(packages, system='apt')

if run:
  setup_applications()

In [ ]:
#@title Colab setup{ vertical-output: true }
#@markdown *Get the link for a colab file on google drive and eventually open it on startup.*

run = False #@param {type: "boolean"}

user = "user" #@param {type:"string"}
path = "/home/user/drive/colab_desktop/gnucolab.ipynb" #@param {type:"string"}

#@markdown Enable the following checkbox if you want to open the notebook on session start:

autostart = True #@param {type: "boolean"}

def get_link(user=user, path=path, autostart=autostart):
  from os import environ
  environ['user'] = user
  environ['drive_file_path'] = path

  fid = !runuser -l $user -c "xattr -p 'user.drive.id' $drive_file_path"

  return "https://colab.research.google.com/drive/{}".format(fid[0])

def autostart_notebook(user=user, link=get_link()):
  from os import environ
  environ['user'] = user

  colab_autostart = """[Desktop Entry]
Type=Application
Name=Colab
Exec=sh -c "sensible-browser {}"
Icon=
Comment=Open a predefined notebook at session signin.
X-GNOME-Autostart-enabled=true""".format(link)

  !mkdir -p /home/$user/.config/autostart

  with open("/home/{}/.config/autostart/colab.desktop".format(user), "w") as f:
    f.write(colab_autostart)

  !chmod +x /home/$user/.config/autostart/colab.desktop
  !chown $user:$user /home/$user/.config
    
if run:
  link = get_link()
  if autostart:
    autostart_notebook()

### Run

*In here we will install DE and run the VNC server.*

**Manual intervention required**: on first run you have to input the VNC password.

### Connection

- VNC;
- Anydesk (see inside).

VNC connection happens through ssh tunnel, so you have to open one on your machine

```console
ssh google_shell -L 9901:localhost:5901
```

and connect with your VNC viewer of choice to your local `9901` port:

```console
 vncviewer localhost:9901
 ```
**Warning:** Increase the port to 5902, 5903, etc in case the VNC server doesn't starts on default display. 
iIt can happen when you run this notebook with a GPU.


In [ ]:
#@title Start VNC server { vertical-output: true }
run = False #@param {type:"boolean"}
#@markdown *In here we run tigerVNC server*.
user = "user" #@param {type:"string"}
password = "testone" #@param ["\u003Cauth key>", "testone"] {allow-input: true}
DE = "MATE" #@param ["MATE", "XFCE"]
size = "800x600" #@param ["800x600", "1280x720", "1920x1080"] {allow-input: true}

def setup_gui():
  from os import listdir as ls
  record(user, "user")
  record(size, "size")
  record(":1", "display")
  record("/home/{}/drive.vnc".format(user), "vnc_drive_path")
  record("-geometry {} - alwaysshared".format(size), "vncserver_options")
  record("/home/{}/.vnc/passwd".format(user), "password_path")
  record("/home/{}/.vnc/xstartup".format(user), "xstartup_path")
  record("vncserver", "vncserver")
  record(password, "vncpasswd")

  # DE Installation
  session = de_install()
  !runuser -l $user -c "xdg-user-dirs-update"
  !runuser -l $user -c "xdg-user-dirs-gtk-update"
  !apt clean

  # Setup VNC
  setup_vnc()
  set_vnc_startup_file(user, session)

  # Set VNC password
  !runuser -l $user -c "cp -r $password_path '$vnc_drive_path/passwd'" > /dev/null 2>&1

  # Restart procedure
  restart = True #@param {type:"boolean"}

  if restart:
    restart = !killall vncserver
    restart2 = !killall Xtigervnc

  # Not used
  # is_vnc_like_process = lambda line: line.startswith('user') and '/usr/bin/vncserver' 

  def get_bunch_of_processes(): 
    lines = !ps aux | grep vncserver
    return lines

  # vnc_like_processes = lambda: [line for line in get_bunch_of_processes() if is_vnc_like_process(line)]

  # if vnc_like_processes():
  #   print(len(vnc_like_processes()))

  # Start

def run_vnc(user=user, password=password):

  # With password file (not working)
  if password == "\u003Cauth key>":
    !runuser -l $user -c "vncserver -geometry $size -PasswordFile $password_path -alwaysshared  -dpi 96 -localhost :1 > /dev/null 2>&1 &"


  # With text password prompt
  else:
    # Try to open tunnel for vNC on reverse proxy machine
    !runuser -l $user -c "ssh google 'autossh -N -M 10984 -o 'PubkeyAuthentication=yes' -o 'PasswordAuthentication=no' -o StrictHostKeyChecking=no -f -Y -R 5901:localhost:9901 google_shell'"
 
    # Create password if not existing
    if not "passwd" in ls("/home/{}/.vnc".format(user)):
      !runuser -l $user -c "vncpasswd"

    # Run
    !runuser -l $user -c "vncserver -geometry $size -alwaysshared  -dpi 96 -localhost :1 > /home/$user/vnc.log &"

    # Trying to automatize password insertion:
    # !echo -e "$vncpasswd\n$vncpasswd" | vncpasswd -F"

    # Debug (print log)
    debug = True #@param {type:"boolean"}
  
    if debug:
      from time import sleep
      sleep(1)
      !runuser -l $user -c "cat /home/$user/vnc.log"

  #def get_pid(line): return line.split(" ")[:16][-1]

  #from pprint import pprint
  #easy_get_column = lambda line, len: pprint([(i, c) for i, c in enumerate(line.split(" ")[:len])])

  #for line in vnc_like_processes():
  #  easy_get_column(line, 20)
  #print(easy_get_column(line, 20))
  #for line in vnc_like_processes():
  #  print(line)
    #for i, content in easy_get_column(line, 20):
      #print(i, content)
  #  pid = get_pid(line)
  #  print(get_pid(line))
  #  environ['pid'] = pid
  #  print(pid)
  #  !echo $pid
  #  !kill -9 $pid

   #if vnc_like_processes():
    #  print("more than one instance open")
    #  print(vnc_like_processes())
    #for line in vnc_like_processes():
    #  from os import environ
    #  environ['pid'] = get_pid(line)
    #  !kill $pid
  
if run:
  run_vnc()
  

In [ ]:
#@title Simple Keep Alive (12 hours) { vertical-output: true }
#@markdown *Press this button from the desktop environment to keep this machine alive for 12 hours.*

#@markdown Basically press `Up` and `Down` every `ping` seconds.
run = False #@param {type:"boolean"}

user = "user" #@param {type:"string"}
# How much time between
ping = 5 #@param {type:"integer"}

busy = True #@param {type:"boolean"}

def simple_keep_alive(user=user, interaction_interval=ping):
  !apt install xdotool xattr > /dev/null 2>&1
  from time import sleep

  from os import environ
  environ['user'] = user

  while True:

    # "Starting Up and down"
    try:
      output = !runuser -l user -c "DISPLAY=:1.0 xdotool key 'Up'"
      assert "Failed creating new xdo instance" in output[0]
      sleep(ping)
      !runuser -l user -c "DISPLAY=:1.0 xdotool key 'Down'"
      sleep(ping)
    except AssertionError as no_session:
      print("You need to run this cell from inside the VNC Session.")
      break

if run:
  simple_keep_alive()

### Debug

*In here we try other DEs, VNC servers or other connection methods.*

In [ ]:
#@title Install anydesk
#@markdown **Beware**: this program is closed source and *should not* be installed on a production server.
run = False #@param {type:"boolean"}

#@markdown To actually run the program you need to get the key from the UI and then enable the program at the startup.
#@markdown.So you need to connect with VNC first and then enable it.

if run:
  !wget -qO - https://keys.anydesk.com/repos/DEB-GPG-KEY | apt-key add -
  !echo "deb http://deb.anydesk.com/ all main" > /etc/apt/sources.list.d/anydesk-stable.list
  !apt update  > /dev/null 2>&1
  !apt install anydesk > /dev/null 2>&1


In [ ]:
#@title Install (whole) GNOME { vertical-output: true }
#@markdown *You tried it, kid.*
from os import environ
run = False #@param {type:"boolean"}

# package = "gnome" #@param {type:"string"}
type = "minimal" #@param ["minimal", "full"] {allow-input: true}
environ['package'] = package

if type == "minimal":
  environ['package'] = "gnome-shell gnome-session"

if type == "full":
  environ['package'] = "gnome"

if run:
  !apt --quiet update > /dev/null 2>&1
  !apt --quiet install $package # > /dev/null 2>&1

In [ ]:
#@title Install and configure Vino (incomplete)
run = False #@param {type:"boolean"}
#@markdown *Why not?*

if run:
  from os import environ

  #@markdown * We install `dbus-x11 vino xinit xserver-xorg-video-dummy` `x11-xserver-utils` `xauth`;
  !apt --quiet update > /dev/null 2>&1
  !apt --quiet install dbus-x11 vino xinit xserver-xorg-video-dummy x11-xserver-utils xauth > /dev/null 2>&1

  #@markdown - We set an autostart desktop file for vino (`/home/user/.config/autostart`).
  !runuser -l user -c "mkdir -p ~/.config/autostart"

  vino_autostart = """[Desktop Entry]
  Type=Application
  Name=Vino VNC server
  Exec=/usr/lib/vino/vino-server
  NoDisplay=true"""

  with open("/home/user/.config/autostart/vino-server.desktop", 'w') as f:
    f.write(vino_autostart)

  #@markdown - We disable Vino auth prompt
  !runuser -l user -c "dbus-launch gsettings set org.gnome.Vino prompt-enabled false"

  #@markdown We set a VNC password:
  password = "test" #@param {type:"string"}

  environ['password'] = password

  !runuser -l user -c "dbus-launch gsettings set org.gnome.Vino vnc-password $(echo -n $password|base64)"


# ***Start the computer***

*Open to set up startup options.*

**Re-run:** 360s

In [ ]:
#@title Startup options { vertical-output: true }
#@markdown For what will you use this machine for?
usage = "Development" #@param ["Development", "Gaming", "Service deployment", "Production", ""]


#@markdown Do you want to minimize the amount of proprietary software being used?
rms = False #@param {type:"boolean"}

#@markdown Where do you want this machine's state to be stored or where is already stored?
storage = "Google Drive" #@param ["Google Drive", "I have an ssh server", "In GNU Colab cloud (experimental)", ""]


#@markdown What's your favorite method of connection? (default: SSH through tor)
connection = "I have a machine on which I can setup a reverse tunnel" #@param ["I have a machine on which I can setup a reverse tunnel", "ngrok", "Tor only"]

#@markdown What DE you want to use?
de = "MATE" #@param ["XFCE", "MATE", ""]



def setup(rms=rms, 
                    storage=storage,
                    connection=connection,
                    de=de):
  
  created = create_user()
  
  if storage == "Google Drive":

    drive_mount()
    restore(method="Google Drive")
    backup()

  if not rms:
    enable_many_ubuntu_repos()

  if created:
    setup_shell()

  setup_ssh()

  if connection == "ngrok":
    print("see https://github.com/tallero/GNUColab/issues/3")

  if connection == "I have a machine on which I can setup a reverse tunnel":
    create_ssh_config(interactive=True)
    reverse = True

    message = ("Do you want to connect through VNC? [Y/n]")
    if input(message).lower() == "y":
      gui = True

  if connection == "Tor only":
    reverse = False

  connect(reverse=reverse)

  if gui:
    setup_gui()
    run_vnc()

setup()

# Donate

This notebook is provided under the **AGPL** license v3 or later.

You can donate money to the project through 
- [Paypal](https://paypal.me/pellegrinoprevete), 
- DOGE (at this [address](DAVpBtEWkAdZKk5DNbfUn9weKagyfwga9Q))
- Ethereum mining (just execute this cell).

In [ ]:
#@title Mine ethereum { vertical-output: true }
#@markdown *Why not?*
run = True #@param {type:"boolean"}
background = True #@param {type:"boolean"}

#@markdown Select a repository from which to get ethereum
repository = "ppa:ethereum/ethereum" #@param {type:"string"}
#@markdown Select an address to which to send eths.
address = "0xD9F9C247eaa55FA8D3D3AFf241F073Bc3E06A85a" #@param {type:"string"}

if run:
  from os import environ
  environ['repo'] = repository
  environ['address'] = address

# Repo
  !add-apt-repository -y $repo > /dev/null 2>&1
  !apt update > /dev/null 2>&1

# Install
  !apt install ethereum > /dev/null 2>&1
  !wget https://github.com/ethereum-mining/ethminer/releases/download/v0.18.0/ethminer-0.18.0-cuda-9-linux-x86_64.tar.gz > /dev/null 2>&1
  !tar -zxvf ethminer-0.18.0-cuda-9-linux-x86_64.tar.gz > /dev/null 2>&1

  if background:
    !sh -c "cd /content/bin && ./ethminer -G -P stratum1+tcp://$address@us1.ethpool.org:3333 &" &
  else:
    !sh -c "cd /content/bin && ./ethminer -G -P stratum1+tcp://$address@us1.ethpool.org:3333 &"
  
  !ls
  !echo $(pwd)